## Decorators & Return Values

- A decorator’s wrapper replaces the original function, so if it forgets to return the original result the caller receives `None`.  
- Many real‑world functions produce critical data (e.g. status strings, dictionaries, numeric results); the decorator must be **transparent** about that value.  
- Fixing this means capturing the result of `func(*args, **kwargs)` inside the wrapper and returning it unchanged.  

In [1]:
def log_calls_broken(func):
    def wrapper(*args, **kwargs):
        print(f"LOG: Calling {func.__name__}")
        func(*args, **kwargs)
        print(f"LOG: Finished {func.__name__}")
    return wrapper

@log_calls_broken
def add(x,y):
    return x + y

print(f"Result seen by caller: {add(2,3)}")    

LOG: Calling add
LOG: Finished add
Result seen by caller: None


## The Wrapper’s Responsibility

- The wrapper is the public face of the decorated function; it must faithfully:  
  - Call the original with all arguments.  
  - Capture its return value.  
  - Perform any extra behaviour (log, time, validate).  
  - **Return** the captured value so callers remain unaware of the wrapper.  
- Failure to return breaks contracts and causes subtle bugs.

### Capturing return values
- Capturing is a one‑liner: `value = func(*args, **kwargs)`.  
- After post‑call logic, `return value` preserves behaviour.  
- You can also inspect or transform `value` before returning if the decorator’s purpose demands it.   

In [3]:
def log_calls_broken(func):
    def wrapper(*args, **kwargs):
        print(f"LOG: Calling {func.__name__}")
        value = func(*args, **kwargs)
        print(f"LOG: Finished {func.__name__}")
        return value
    return wrapper

@log_calls_broken
def add(x,y):
    return x + y

print(f"Result seen by caller: {add(2,3)}")    

LOG: Calling add
LOG: Finished add
Result seen by caller: 5


## Handling Exceptions in Decorators

- Wrappers often log exceptions for observability but should **re‑raise** them so callers can still handle or see errors.  
- Use `try ... except ... raise` around the call; log inside the `except`, then re‑raise without arguments to preserve traceback.  
- A decorator that swallows exceptions changes program semantics unless that is its explicit purpose (e.g. retry).  

In [ ]:
def log_and_reraise(func):
    def wrapper(*args, **kwargs):
        try:
            return func(*args, **kwargs)
        except Exception as err:
            print(f"[ERROR] {func.__name__} raised {err.__class__.__name__}")
            
    return wrapper

@log_and_reraise
def fail():
    raise ValueError("simulated problem")

fail()    



[ERROR] fail raised ValueError


It prints the exception but no exception and traceback is shown. The correct way is to use raise.

In [6]:
def log_and_reraise(func):
    def wrapper(*args, **kwargs):
        try:
            return func(*args, **kwargs)
        except Exception as err:
            print(f"[ERROR] {func.__name__} raised {err.__class__.__name__}")
            raise
    return wrapper

@log_and_reraise
def fail():
    raise ValueError("simulated problem")

fail()    



[ERROR] fail raised ValueError


ValueError: simulated problem